In [24]:
import platform 
print("Python: {} ".format(platform.python_version()))
import numpy as np 
import pandas as pd
import xgboost as xgb

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestRegressor 

Python: 3.6.0 


In [25]:
people = pd.read_csv("people.csv", dtype={'people_id': np.str, 'activity_id': np.str, 'char_38': np.int32}, parse_dates=['date'])
people[:1230]
people.head()

,people_id,char_1,group_1,char_2,date,char_3,char_4,char_5,char_6,char_7,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,ppl_100,type 2,group 17304,type 2,2021-06-29,type 5,type 5,type 5,type 3,type 11,...,False,True,True,False,False,True,True,True,False,36
1,ppl_100002,type 2,group 8688,type 3,2021-01-06,type 28,type 9,type 5,type 3,type 11,...,False,True,True,True,True,True,True,True,False,76
2,ppl_100003,type 2,group 33592,type 3,2022-06-10,type 4,type 8,type 5,type 2,type 5,...,False,False,True,True,True,True,False,True,True,99
3,ppl_100004,type 2,group 22593,type 3,2022-07-20,type 40,type 25,type 9,type 4,type 16,...,True,True,True,True,True,True,True,True,True,76
4,ppl_100006,type 2,group 6534,type 3,2022-07-27,type 40,type 25,type 9,type 3,type 8,...,False,False,True,False,False,False,True,True,False,84


In [26]:
train = pd.read_csv("act_train.csv",dtype={'people_id': np.str, 'activity_id': np.str, 'outcome': np.int8}, parse_dates=['date'])
train[:1230]
train.head()

,people_id,activity_id,date,activity_category,char_1,char_2,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10,outcome
0,ppl_100,act2_1734928,2023-08-26,type 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 76,0
1,ppl_100,act2_2434093,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
2,ppl_100,act2_3404049,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
3,ppl_100,act2_3651215,2023-08-04,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
4,ppl_100,act2_4109017,2023-08-26,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0


In [27]:
test = pd.read_csv("act_test.csv",dtype={'people_id': np.str, 'activity_id': np.str}, parse_dates=['date'])
test[:1230]
test.head()

,people_id,activity_id,date,activity_category,char_1,char_2,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10
0,ppl_100004,act1_249281,2022-07-20,type 1,type 5,type 10,type 5,type 1,type 6,type 1,type 1,type 7,type 4,NaN
1,ppl_100004,act2_230855,2022-07-20,type 5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 682
2,ppl_10001,act1_240724,2022-10-14,type 1,type 12,type 1,type 5,type 4,type 6,type 1,type 1,type 13,type 10,NaN
3,ppl_10001,act1_83552,2022-11-27,type 1,type 20,type 10,type 5,type 4,type 6,type 1,type 1,type 5,type 5,NaN
4,ppl_10001,act2_1043301,2022-10-15,type 5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 3015


In [28]:
people.describe()

,char_38
count,189118.000000
mean,50.327399
std,35.752127
min,0.000000
25%,10.000000
50%,58.000000
75%,83.000000
max,100.000000


In [29]:
train.describe()

,outcome
count,2.197291e+06
mean,4.439544e-01
std,4.968491e-01
min,0.000000e+00
25%,0.000000e+00
50%,0.000000e+00
75%,1.000000e+00
max,1.000000e+00


In [30]:
test.describe()

,people_id,activity_id,date,activity_category,char_1,char_2,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10
count,498687,498687,498687,498687,40092,40092,40092,40092,40092,40092,40092,40092,40092,458595
unique,37823,498687,411,7,48,31,11,7,6,5,8,18,19,3961
top,ppl_112017,act2_3837226,2022-09-16 00:00:00,type 2,type 2,type 2,type 1,type 3,type 6,type 2,type 1,type 4,type 8,type 1
freq,1650,1,10302,223164,9862,12757,9927,25046,17131,15940,13290,19803,7888,223164
first,NaN,NaN,2022-07-17 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,2023-08-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
people.columns.difference(test.columns)

Index(['char_11', 'char_12', 'char_13', 'char_14', 'char_15', 'char_16',
       'char_17', 'char_18', 'char_19', 'char_20', 'char_21', 'char_22',
       'char_23', 'char_24', 'char_25', 'char_26', 'char_27', 'char_28',
       'char_29', 'char_30', 'char_31', 'char_32', 'char_33', 'char_34',
       'char_35', 'char_36', 'char_37', 'char_38', 'group_1'],
      dtype='object')

In [32]:
print ('Merge Data ')

Merge Data 


In [33]:
X_train = train.merge(people, on='people_id', how='left', left_index=True)
X_test  = test.merge(people, on='people_id', how='left', left_index=True)

In [34]:
X_train[:1230]
X_train.head();

In [35]:
list(X_train)

['people_id',
 'activity_id',
 'date_x',
 'activity_category',
 'char_1_x',
 'char_2_x',
 'char_3_x',
 'char_4_x',
 'char_5_x',
 'char_6_x',
 'char_7_x',
 'char_8_x',
 'char_9_x',
 'char_10_x',
 'outcome',
 'char_1_y',
 'group_1',
 'char_2_y',
 'date_y',
 'char_3_y',
 'char_4_y',
 'char_5_y',
 'char_6_y',
 'char_7_y',
 'char_8_y',
 'char_9_y',
 'char_10_y',
 'char_11',
 'char_12',
 'char_13',
 'char_14',
 'char_15',
 'char_16',
 'char_17',
 'char_18',
 'char_19',
 'char_20',
 'char_21',
 'char_22',
 'char_23',
 'char_24',
 'char_25',
 'char_26',
 'char_27',
 'char_28',
 'char_29',
 'char_30',
 'char_31',
 'char_32',
 'char_33',
 'char_34',
 'char_35',
 'char_36',
 'char_37',
 'char_38']

In [36]:
X_train.shape

(2197291, 55)

In [37]:
# outcome is the y value of the training set

# char_10_y, char_11 => char_37 are boolean
# char_38 are numerical => no need to do anything
# char_1_y => char_9_y are categorical => do a one-hot encoding to get sparse matrix
# char_1_x => char_10_x are categorical => do a one-hot encoding to get sparse matrix
# group_1 is categorical
# date_x is the activity date, date_y is possibly the user registration date.
# activity_categorical is categorical
# people_id and activity_id will not contribute to the decision => can be used as index

In [38]:
used_separately = ['people_id', 'activity_id', 'date_x', 'date_y', 'outcome', 'char_38']
categorical = ['char_1_y', 'char_2_y', 'char_3_y', 'char_4_y', 'char_5_y',
               'char_6_y', 'char_7_y', 'char_8_y', 'char_9_y',
               'char_1_x', 'char_2_x', 'char_3_x', 'char_4_x', 'char_5_x',
               'char_6_x', 'char_7_x', 'char_8_x', 'char_9_x', 'char_10_x',
               'group_1', 'activity_category']
not_categorical = [x for x in X_train.columns if x not in categorical \
                   and x not in used_separately]
not_categorical

['char_10_y',
 'char_11',
 'char_12',
 'char_13',
 'char_14',
 'char_15',
 'char_16',
 'char_17',
 'char_18',
 'char_19',
 'char_20',
 'char_21',
 'char_22',
 'char_23',
 'char_24',
 'char_25',
 'char_26',
 'char_27',
 'char_28',
 'char_29',
 'char_30',
 'char_31',
 'char_32',
 'char_33',
 'char_34',
 'char_35',
 'char_36',
 'char_37']

In [39]:
print ('Preprocessing Function')

Preprocessing Function


In [40]:
def preprocessing(dataset):
    local_data = dataset
    
    for col in list(local_data.columns):
        # fill the categorical NA with type 0,
        # for each categorical col, we only take the number, and convert to int32
        if col in categorical:
            local_data[col].fillna('type 0', inplace=True)
            local_data[col] = local_data[col].apply(
                lambda x: x.split(' ')[1]).astype(np.int32)

        if col in not_categorical:
            # boolean values are converted to int aswell
            local_data[col] = local_data[col].astype(np.int8)
        # this doesnot deal with date time data
    
    # date_x is the activity date, date_y is possibly the reg date.
    local_data['year_x'] = local_data['date_x'].dt.year
    local_data['month_x'] = local_data['date_x'].dt.month
    local_data['day_x'] = local_data['date_x'].dt.day
    local_data['isweekend_x'] = (local_data['date_x'].dt.weekday >= 5).astype(int)
    # drop date_x after splitting its features
    local_data = local_data.drop('date_x', axis = 1)
    # we may not need date_y as its only reg date
    local_data = local_data.drop('date_y', axis = 1)
    return local_data

In [41]:
X_train = preprocessing(X_train)
X_train[:1230]
X_train.head()

,people_id,activity_id,activity_category,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,char_7_x,...,char_33,char_34,char_35,char_36,char_37,char_38,year_x,month_x,day_x,isweekend_x
0,ppl_100,act2_1734928,4,0,0,0,0,0,0,0,...,0,1,1,1,0,36,2023,8,26,1
0,ppl_100,act2_2434093,2,0,0,0,0,0,0,0,...,0,1,1,1,0,36,2022,9,27,0
0,ppl_100,act2_3404049,2,0,0,0,0,0,0,0,...,0,1,1,1,0,36,2022,9,27,0
0,ppl_100,act2_3651215,2,0,0,0,0,0,0,0,...,0,1,1,1,0,36,2023,8,4,0
0,ppl_100,act2_4109017,2,0,0,0,0,0,0,0,...,0,1,1,1,0,36,2023,8,26,1


In [42]:
X_train=X_train.sort_values(['people_id'], ascending=[1])
X_train[:1230]
X_train.head()

,people_id,activity_id,activity_category,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,char_7_x,...,char_33,char_34,char_35,char_36,char_37,char_38,year_x,month_x,day_x,isweekend_x
0,ppl_100,act2_1734928,4,0,0,0,0,0,0,0,...,0,1,1,1,0,36,2023,8,26,1
0,ppl_100,act2_2434093,2,0,0,0,0,0,0,0,...,0,1,1,1,0,36,2022,9,27,0
0,ppl_100,act2_3404049,2,0,0,0,0,0,0,0,...,0,1,1,1,0,36,2022,9,27,0
0,ppl_100,act2_3651215,2,0,0,0,0,0,0,0,...,0,1,1,1,0,36,2023,8,4,0
0,ppl_100,act2_4109017,2,0,0,0,0,0,0,0,...,0,1,1,1,0,36,2023,8,26,1


In [43]:
X_test = preprocessing(X_test)
X_test[:1230]
X_test.head()

,people_id,activity_id,activity_category,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,char_7_x,...,char_33,char_34,char_35,char_36,char_37,char_38,year_x,month_x,day_x,isweekend_x
3,ppl_100004,act1_249281,1,5,10,5,1,6,1,1,...,1,1,1,1,1,76,2022,7,20,0
3,ppl_100004,act2_230855,5,0,0,0,0,0,0,0,...,1,1,1,1,1,76,2022,7,20,0
5,ppl_10001,act1_240724,1,12,1,5,4,6,1,1,...,1,1,1,1,1,90,2022,10,14,0
5,ppl_10001,act1_83552,1,20,10,5,4,6,1,1,...,1,1,1,1,1,90,2022,11,27,1
5,ppl_10001,act2_1043301,5,0,0,0,0,0,0,0,...,1,1,1,1,1,90,2022,10,15,1


In [44]:
X_test=X_test.sort_values(['people_id'], ascending=[1])
X_test[:1230]
X_test.head()

,people_id,activity_id,activity_category,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,char_7_x,...,char_33,char_34,char_35,char_36,char_37,char_38,year_x,month_x,day_x,isweekend_x
3,ppl_100004,act1_249281,1,5,10,5,1,6,1,1,...,1,1,1,1,1,76,2022,7,20,0
3,ppl_100004,act2_230855,5,0,0,0,0,0,0,0,...,1,1,1,1,1,76,2022,7,20,0
5,ppl_10001,act2_688604,5,0,0,0,0,0,0,0,...,1,1,1,1,1,90,2022,11,28,0
5,ppl_10001,act2_659237,5,0,0,0,0,0,0,0,...,1,1,1,1,1,90,2022,10,16,1
5,ppl_10001,act2_649143,5,0,0,0,0,0,0,0,...,1,1,1,1,1,90,2022,11,27,1
5,ppl_10001,act2_596561,5,0,0,0,0,0,0,0,...,1,1,1,1,1,90,2022,10,15,1
5,ppl_10001,act2_584216,5,0,0,0,0,0,0,0,...,1,1,1,1,1,90,2022,11,27,1
5,ppl_10001,act2_4649285,5,0,0,0,0,0,0,0,...,1,1,1,1,1,90,2022,10,15,1
5,ppl_10001,act2_4397847,5,0,0,0,0,0,0,0,...,1,1,1,1,1,90,2022,10,15,1
5,ppl_10001,act2_4167474,5,0,0,0,0,0,0,0,...,1,1,1,1,1,90,2022,10,15,1


In [45]:
y_train = X_train.outcome
X_train=X_train.drop('outcome',axis=1)
y_train[:1230]

0      0
0      0
0      0
0      0
0      0
0      0
1      1
1      1
2      1
2      1
2      1
2      1
2      1
2      1
2      1
2      1
2      1
2      1
2      1
2      1
2      1
2      1
2      1
2      1
2      1
2      1
2      1
2      1
2      1
2      1
      ..
135    1
135    1
135    1
135    1
135    1
135    1
135    1
135    1
135    1
135    1
135    1
135    1
135    1
135    1
135    1
135    1
135    1
135    1
135    1
135    1
135    1
135    1
135    1
135    1
135    1
135    1
135    1
135    1
135    1
135    1
Name: outcome, dtype: int8

In [48]:
# now we need to take care of categorial data using one-hot-encoding.
# but the number of dimension resulted using one-hot-ending will equal the max number 
# in the categorical data.

# thus we need to reduce the number of dimension by checking only those numbers that
# appear in either train or test set.

# the following is a cheat because we're not supposed to use the data from the 
# test set for the preprocessing of the training set

# join the two set so the one hot encoding will have all the values for each categorical
# column
All = pd.concat([X_train, X_test], ignore_index=True)

In [49]:
All[categorical][:1230]
All[categorical].head()

,char_1_y,char_2_y,char_3_y,char_4_y,char_5_y,char_6_y,char_7_y,char_8_y,char_9_y,char_1_x,...,char_3_x,char_4_x,char_5_x,char_6_x,char_7_x,char_8_x,char_9_x,char_10_x,group_1,activity_category
0,2,2,5,5,5,3,11,2,2,0,...,0,0,0,0,0,0,0,76,17304,4
1,2,2,5,5,5,3,11,2,2,0,...,0,0,0,0,0,0,0,1,17304,2
2,2,2,5,5,5,3,11,2,2,0,...,0,0,0,0,0,0,0,1,17304,2
3,2,2,5,5,5,3,11,2,2,0,...,0,0,0,0,0,0,0,1,17304,2
4,2,2,5,5,5,3,11,2,2,0,...,0,0,0,0,0,0,0,1,17304,2
5,2,2,5,5,5,3,11,2,2,0,...,0,0,0,0,0,0,0,1727,17304,4
6,2,3,28,9,5,3,11,2,4,0,...,0,0,0,0,0,0,0,1,8688,2
7,2,3,28,9,5,3,11,2,4,0,...,0,0,0,0,0,0,0,1,8688,2
8,2,3,4,8,5,2,5,2,2,0,...,0,0,0,0,0,0,0,276,33592,4
9,2,3,4,8,5,2,5,2,2,0,...,0,0,0,0,0,0,0,1,33592,2


In [50]:
# then fit the one hot encoding using both sets
enc = OneHotEncoder(handle_unknown='ignore')
enc = enc.fit(All[categorical])

X_train_ohe=enc.transform(X_train[categorical])
X_train_ohe[:1230]

<1230x41499 sparse matrix of type '<class 'numpy.float64'>'
	with 25830 stored elements in Compressed Sparse Row format>

In [51]:
X_test_ohe=enc.transform(X_test[categorical])
X_test_ohe.shape

(498687, 41499)

In [52]:
not_categorical.extend(['char_38', 'year_x',
       'month_x', 'day_x', 'isweekend_x'])
not_categorical

['char_10_y',
 'char_11',
 'char_12',
 'char_13',
 'char_14',
 'char_15',
 'char_16',
 'char_17',
 'char_18',
 'char_19',
 'char_20',
 'char_21',
 'char_22',
 'char_23',
 'char_24',
 'char_25',
 'char_26',
 'char_27',
 'char_28',
 'char_29',
 'char_30',
 'char_31',
 'char_32',
 'char_33',
 'char_34',
 'char_35',
 'char_36',
 'char_37',
 'char_38',
 'year_x',
 'month_x',
 'day_x',
 'isweekend_x']

In [53]:
All.columns

Index(['people_id', 'activity_id', 'activity_category', 'char_1_x', 'char_2_x',
       'char_3_x', 'char_4_x', 'char_5_x', 'char_6_x', 'char_7_x', 'char_8_x',
       'char_9_x', 'char_10_x', 'char_1_y', 'group_1', 'char_2_y', 'char_3_y',
       'char_4_y', 'char_5_y', 'char_6_y', 'char_7_y', 'char_8_y', 'char_9_y',
       'char_10_y', 'char_11', 'char_12', 'char_13', 'char_14', 'char_15',
       'char_16', 'char_17', 'char_18', 'char_19', 'char_20', 'char_21',
       'char_22', 'char_23', 'char_24', 'char_25', 'char_26', 'char_27',
       'char_28', 'char_29', 'char_30', 'char_31', 'char_32', 'char_33',
       'char_34', 'char_35', 'char_36', 'char_37', 'char_38', 'year_x',
       'month_x', 'day_x', 'isweekend_x'],
      dtype='object')

In [54]:
from scipy.sparse import hstack

In [55]:
X_train_all = hstack((X_train[not_categorical], X_train_ohe))
X_train_all.shape

(2197291, 41532)

In [56]:
X_test_all = hstack((X_test[not_categorical], X_test_ohe))
X_test_all.shape

(498687, 41532)

In [57]:
print("Training data: " + format(X_train_all.shape))
print("Test data: " + format(X_test_all.shape))
print("###########")
print("One Hot enconded Test Dataset Script")

Training data: (2197291, 41532)
Test data: (498687, 41532)
###########
One Hot enconded Test Dataset Script


In [58]:
print ('running cross validation')

running cross validation


In [59]:
dtrain = xgb.DMatrix(X_train_all,label=y_train)
dtest = xgb.DMatrix(X_test_all)

param = {'max_depth':2, 'eta':1, 'silent':1, 'objective':'binary:logistic'}
num_round = 2

AttributeError: module 'xgboost' has no attribute 'DMatrix'

In [ ]:
# do cross validation, this will print result out as
# [iteration]  metric_name:mean_value+std_value
# std_value is standard deviation of the metric

xgb.cv(param, dtrain, num_round, nfold=5,
       metrics={'error'}, seed = 0,
       callbacks=[xgb.callback.print_evaluation(show_stdv=True)])

In [ ]:
print ('running cross validation, disable standard deviation display')
# do cross validation, this will print result out as
# [iteration]  metric_name:mean_value

In [ ]:
res = xgb.cv(param, dtrain, num_boost_round=10, nfold=5,
             metrics={'error'}, seed = 0,
             callbacks=[xgb.callback.print_evaluation(show_stdv=False),
                        xgb.callback.early_stop(3)])

In [ ]:
print (res)

In [ ]:
print ('running cross validation, with preprocessing function')

In [ ]:
# define the preprocessing function
# used to return the preprocessed training, test data, and parameter
# we can use this to do weight rescale, etc.
# as a example, we try to set scale_pos_weight
def fpreproc(dtrain, dtest, param):
    label = dtrain.get_label()
    ratio = float(np.sum(label == 0)) / np.sum(label==1)
    param['scale_pos_weight'] = ratio
    return (dtrain, dtest, param)

In [ ]:
# do cross validation, for each fold
# the dtrain, dtest, param will be passed into fpreproc
# then the return value of fpreproc will be used to generate
# results of that fold
xgb.cv(param, dtrain, num_round, nfold=5,
       metrics={'auc'}, seed = 0, fpreproc = fpreproc)

In [ ]:
###
# you can also do cross validation with cutomized loss function
# See custom_objective.py
##
print ('running cross validation, with cutomsized loss function')

In [ ]:
def logregobj(preds, dtrain):
    labels = dtrain.get_label()
    preds = 1.0 / (1.0 + np.exp(-preds))
    grad = preds - labels
    hess = preds * (1.0-preds)
    return grad, hess

In [ ]:
def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'error', float(sum(labels != (preds > 0.0))) / len(labels)

In [ ]:
param = {'max_depth':2, 'eta':1, 'silent':1}

In [ ]:
# train with customized objective
xgb.cv(param, dtrain, num_round, nfold = 5, seed = 0,
       obj = logregobj, feval=evalerror)